In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch, evaluate, os
import torch.nn as nn
import pandas as pd

/home/huynv/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
project_folder = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
data_folder = os.path.join(project_folder, 'data')
checkpoint_folder = os.path.join(project_folder, 'checkpoint')
print(project_folder, data_folder, checkpoint_folder)

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def post_processing(text):
    text = text.replace("</s>", " ")
    text = text.replace("<s>", " ")
    text = text.replace("<unk>", " ")
    text = text.replace("<pad>", " ")
    text = text.replace("_", " ")
    text = " ".join(text.strip().split()).lower()
    return text

In [4]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable") 
model = AutoModelForSeq2SeqLM.from_pretrained("vinai/bartpho-syllable").to(device)
rouge = evaluate.load('rouge')

In [5]:
training_args = Seq2SeqTrainingArguments(checkpoint_folder,
                                      do_train=True,
                                      do_eval=False,
                                      num_train_epochs=2,
                                      learning_rate=1e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=1,
                                      per_device_eval_batch_size=1,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      save_total_limit=3,
                                      fp16=True,
                                      )

In [6]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
)
trainer._load_from_checkpoint(os.path.join(checkpoint_folder , 'checkpoint-50000'))

In [7]:
def gen_text(model, tokenizer, sentence):
    text = sentence + " </s>"
    encoding = tokenizer(text, return_tensors="pt", padding='max_length', pad_to_max_length=True,
                                    max_length=1024, truncation=True, return_token_type_ids=False)
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
    outputs = model.generate(
        input_ids=input_ids, attention_mask=attention_masks, early_stopping=True, max_length=256
    )
    output = tokenizer.decode(outputs.reshape(-1))
    del input_ids, attention_masks, encoding, outputs
    torch.cuda.empty_cache()
    return post_processing(output)

In [8]:
df_test = pd.read_csv(os.path.join(data_folder,'vietnews_test.csv'))
df_test

,Unnamed: 0,file,original,summary
0,0,vietnews-master/data/test_tokenized\009449.txt...,"Đoạn video cho thấy hôm 20/9 , Rakeyia_Scott đ...",Khoảnh_khắc cảnh_sát Mỹ bắn một người đàn_ông ...
1,1,vietnews-master/data/test_tokenized\008285.txt...,Các binh_sĩ Hàn_Quốc tham_gia cuộc tập_trận ch...,Chi_phí tốn_kém là một trong những lý_do thúc_...
2,2,vietnews-master/data/test_tokenized\002060.txt...,"Lên sóng vào năm 2002 , bộ phim Giày thuỷ_tinh...",Ở tuổi 41 nam chính trong phim “ Giày thuỷ_tin...
3,3,vietnews-master/data/test_tokenized\001933.txt...,"Kể từ khi công_khai tình_cảm , Linh_Chi và Lâm...","Nối lại tình xưa sau thời_gian “ đứt_quãng ” ,..."
4,4,vietnews-master/data/test_tokenized\021360.txt...,"Sau khi đẻ rơi trên đường , mẹ con sản_phụ Ngu...","Đang trên đường đến bệnh_viện bằng xe_máy , ch..."
...,...,...,...,...
22639,22639,vietnews-master/data/test_tokenized\010053.txt...,Thủ_tướng Nguyễn_Xuân_Phúc và Thủ_tướng Nhật_B...,Thủ_tướng Nhật_Bản Shinzo_Abe hôm_nay cho_biết...
22640,22640,vietnews-master/data/test_tokenized\008629.txt...,Thứ_trưởng Ngoại_giao Syria_Faisal al - Moqdad...,Syria cảnh_báo mọi cuộc tấn_công của Mỹ trên l...
22641,22641,vietnews-master/data/test_tokenized\003169.txt...,"Nằm trong kế_hoạch di_dời , giải_toả các chung...",UBND quận 1 cưỡng_chế các hộ dân còn lại tại c...
22642,22642,vietnews-master/data/test_tokenized\004993.txt...,"Theo hãng tin Reuters , trong các văn_bản vừa ...",Bộ Ngoại_giao Mỹ muốn yêu_cầu tất_cả ứng_viên ...


In [9]:
preds = []
refs = []
for i in range(len(df_test)):
    row = df_test.iloc[i]
    origin_text = row['original'].lower()
    summary_text = row['summary'].lower().replace("_", " ")
    model_gen = gen_text(trainer.model, tokenizer, origin_text)
    preds.append(model_gen)
    refs.append(summary_text)
    print(f"Model predict {i + 1}: {model_gen}")
    print(f"Reference {i + 1}: {summary_text}")
    if len(refs) % 20 == 0:
        print("_"*100)
        print(f"ROUGE SCORE {len(refs)} samples: {rouge.compute(predictions=preds, references=refs)}")
        print("_"*100)

Model predict 1: một người đàn ông mỹ bị bắn chết khi đang bị cảnh sát truy quét tại một chung cư ở bang north carolina, mỹ.
Reference 1: khoảnh khắc cảnh sát mỹ bắn một người đàn ông da màu đã được vợ của nạn nhân ghi lại khi bà van xin họ đừng bắn chồng mình . 
Model predict 2: mỹ cho biết chi phí cho một cuộc diễn tập người bảo vệ tự do ulchi giữa quân đội nước này và hàn quốc lên tới 14 triệu usd.
Reference 2: chi phí tốn kém là một trong những lý do thúc đẩy trump chấm dứt các cuộc tập trận chung mỹ - hàn . 
Model predict 3: sau nhiều năm " chìm nghỉm " trong showbiz, so ji sub một bước vụt sáng thành sao nhờ vai diễn ấn tượng trong dự án ăn khách giày thuỷ tinh.
Reference 3: ở tuổi 41 nam chính trong phim “ giày thuỷ tinh ” đã gặt hái được nhiều thành công trong sự nghiệp khiến nhiều người phải khao khát , nhưng trong tình duyên anh lại lận đận và vẫn đang miệt mài đi tìm hạnh phúc cho riêng mình . 
Model predict 4: mới đây, chân dài họ trần bất ngờ tiết lộ hình ảnh lãng mạn bên 